<a href="https://colab.research.google.com/github/ss1111119/ChillTrade/blob/main/ShouldISell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import textwrap
import google.generativeai as genai
import pandas as pd
from datetime import datetime, time
import talib as ta

# 顯示當前日期和時間
now = datetime.now()
print(f"當前日期和時間: {now.strftime('%Y-%m-%d %H:%M:%S')}")

# 設定參數
api_key = ''  # 請替換為您的 Gemini API Key
ticker = '2408.TW'  # 要分析的股票代碼
buy_price = 69  # 您的購買價格


# 檢查當前時間是否在市場交易時間內
def is_market_open():
    now = datetime.now()
    market_open_time = time(9, 0)  # 假設市場開盤時間為上午9點
    market_close_time = time(13, 30)  # 假設市場收盤時間為下午1點30分
    return market_open_time <= now.time() <= market_close_time


# 分析股票
def analyze_stock(ticker, buy_price):
    if not is_market_open():
        print("提醒：當前時間不在市場交易時間內，數據可能不完整或不準確。")

    stock = yf.Ticker(ticker)

    # 取得歷史價格數據
    stock_data = stock.history(start="2021-01-01", end=datetime.now().strftime('%Y-%m-%d'))

    # 檢查是否有 'Adj Close' 列，如果沒有則使用 'Close'
    if 'Adj Close' not in stock_data.columns:
        stock_data['Adj Close'] = stock_data['Close']

    # 取得股息和拆股信息
    dividends = stock.dividends
    splits = stock.splits

    # 取得財務報表
    balance_sheet = stock.balance_sheet
    income_statement = stock.financials
    cashflow = stock.cashflow

    # 取得公司信息
    info = stock.info
    market_cap = info.get('marketCap')
    pe_ratio = info.get('trailingPE')
    eps = info.get('trailingEps')
    industry = info.get('industry')
    description = info.get('longBusinessSummary')

    # 取得財務比率
    current_ratio = info.get('currentRatio')
    quick_ratio = info.get('quickRatio')
    profit_margin = info.get('profitMargins')

    # 取得自由現金流
    free_cashflow = info.get('freeCashflow')

    # 取得收益增長率
    earnings_growth = info.get('earningsGrowth')

    # 取得收入增長率
    revenue_growth = info.get('revenueGrowth')

    # 取得淨利潤率
    net_profit_margin = info.get('netProfitMargins')

    # 取得股東權益回報率 (ROE)
    roe = info.get('returnOnEquity')

    # 取得企業價值
    enterprise_value = info.get('enterpriseValue')

    # 取得每股淨資產 (BVPS)
    bvps = info.get('bookValue')

    # 取得分析師推薦
    recommendations = stock.recommendations

    # 取得貝塔係數
    beta = info.get('beta')

    # 取得股東信息（使用 try-except 捕獲異常）
    try:
        major_holders = stock.major_holders
        institutional_holders = stock.institutional_holders
    except Exception as e:
        major_holders = "無法獲取主要股東信息"
        institutional_holders = "無法獲取機構持股信息"
        print(f"Error retrieving holders data: {e}")


    # 從財務報表中取得淨利潤和收入，並手動計算收入增長率和淨利潤率
    try:
        # 手動計算收入增長率
        total_revenue_current = income_statement.loc['Total Revenue'].iloc[0]  # 最近一年的總收入
        total_revenue_previous = income_statement.loc['Total Revenue'].iloc[1]  # 前一年的總收入
        revenue_growth = (total_revenue_current - total_revenue_previous) / total_revenue_previous * 100

        # 計算淨利潤率
        net_income_current = income_statement.loc['Net Income'].iloc[0]  # 最近一年的淨利潤
        net_income_previous = income_statement.loc['Net Income'].iloc[1]  # 前一年的淨利潤
        earnings_growth = (net_income_current - net_income_previous) / net_income_previous * 100
        net_profit_margin = (net_income_current / total_revenue_current) * 100
    except KeyError:
        print("無法取得淨利潤或收入數據，無法計算淨利潤率或增長率。")
        revenue_growth = None
        earnings_growth = None
        net_profit_margin = None


    # 計算技術指標
    stock_data['macd'], stock_data['macd_signal'], stock_data['macd_diff'] = calculate_macd(stock_data['Adj Close'])
    delta = stock_data['Adj Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    stock_data['rsi'] = rsi

    # 計算綜合信號
    stock_data['composite_signal'] = stock_data.apply(lambda x: generate_composite_signal(x['macd'], x['rsi']), axis=1)
    stock_data['vwap'] = calculate_vwap(stock_data)
    stock_data['ad_line'] = calculate_ad_line(stock_data)
    stock_data['volume_oscillator'] = calculate_volume_oscillator(stock_data)
    stock_data['atr'] = calculate_atr(stock_data)
    stock_data['parabolic_sar'] = calculate_parabolic_sar(stock_data)
    stock_data['stochastic_k'], stock_data['stochastic_d'] = calculate_stochastic(stock_data)
    stock_data['vwma'] = calculate_vwma(stock_data)
    stock_data['vroc'] = calculate_vroc(stock_data)
    stock_data['momentum'] = calculate_momentum(stock_data)
    stock_data['williams_r'] = calculate_williams_r(stock_data)
    stock_data['pivot_point'], stock_data['support_1'], stock_data['support_2'], stock_data['resistance_1'], stock_data['resistance_2'] = calculate_pivot_points(stock_data)
    # stock_data['fibonacci_retracement'] = calculate_fibonacci_retracement(stock_data)
    stock_data['rvi'] = calculate_rvi(stock_data)

    stock_data['50_day_ma'] = stock_data['Adj Close'].rolling(window=50).mean()
    stock_data['200_day_ma'] = stock_data['Adj Close'].rolling(window=200).mean()
    stock_data['kd_k'], stock_data['kd_d'] = calculate_kd(stock_data)

    max_price = stock_data['Adj Close'].cummax()
    drawdown = (stock_data['Adj Close'] - max_price) / max_price
    max_drawdown = drawdown.min()

    annual_return = (stock_data['Adj Close'].iloc[-1] / buy_price) ** (1 / 3) - 1

    # 計算布林帶
    stock_data['bb_upper'], stock_data['bb_lower'] = calculate_bollinger_bands(stock_data['Adj Close'])

    # 計算ADX
    stock_data['adx'] = calculate_adx(stock_data)

    # 計算Ichimoku指標
    ichimoku_a, ichimoku_b, kijun_sen, tenkan_sen, chikou_span = calculate_ichimoku(stock_data)
    stock_data['ichimoku_a'] = ichimoku_a
    stock_data['ichimoku_b'] = ichimoku_b

    # 計算市場情緒和行業趨勢
    market_sentiment = get_market_sentiment()
    industry_trend = get_industry_trend()


    # 計算費波那契回撤位
    fibonacci_retracement = calculate_fibonacci_retracement(stock_data)
    if fibonacci_retracement is None:
        print("無法計算費波那契回撤位")

    return {
        'market_sentiment': market_sentiment,
        'industry_trend': industry_trend,
        'composite_signal': stock_data['composite_signal'],
        'vwap': stock_data['vwap'].iloc[-1],
        'ad_line': stock_data['ad_line'].iloc[-1],
        'volume_oscillator': stock_data['volume_oscillator'].iloc[-1],
        'current_price': stock_data['Adj Close'].iloc[-1],
        'atr': stock_data['atr'].iloc[-1],
        'parabolic_sar': stock_data['parabolic_sar'].iloc[-1],
        'stochastic_k': stock_data['stochastic_k'].iloc[-1],
        'stochastic_d': stock_data['stochastic_d'].iloc[-1],
        'vwma': stock_data['vwma'].iloc[-1],
        'vroc': stock_data['vroc'].iloc[-1],
        'momentum': stock_data['momentum'].iloc[-1],
        'williams_r': stock_data['williams_r'].iloc[-1],
        'pivot_point': stock_data['pivot_point'].iloc[-1],
        'support_1': stock_data['support_1'].iloc[-1],
        'support_2': stock_data['support_2'].iloc[-1],
        'resistance_1': stock_data['resistance_1'].iloc[-1],
        'resistance_2': stock_data['resistance_2'].iloc[-1],
        # 'fibonacci_retracement': stock_data['fibonacci_retracement'].iloc[-1],
        'fibonacci_retracement': fibonacci_retracement,
        'rvi': stock_data['rvi'].iloc[-1],
        'max_drawdown': max_drawdown * 100,
        'annual_return': annual_return * 100,
        '50_day_ma': stock_data['50_day_ma'].iloc[-1],
        '200_day_ma': stock_data['200_day_ma'].iloc[-1],
        'rsi': stock_data['rsi'].iloc[-1],
        'macd': stock_data['macd'].iloc[-1],
        'macd_signal': stock_data['macd_signal'].iloc[-1],
        'kd_k': stock_data['kd_k'].iloc[-1],
        'kd_d': stock_data['kd_d'].iloc[-1],
        'bb_upper': stock_data['bb_upper'].iloc[-1],
        'bb_lower': stock_data['bb_lower'].iloc[-1],
        'adx': stock_data['adx'].iloc[-1],
        'ichimoku_a': stock_data['ichimoku_a'].iloc[-1],
        'ichimoku_b': stock_data['ichimoku_b'].iloc[-1],
        'buy_price': buy_price,
        'dividends': dividends,
        'splits': splits,
        'balance_sheet': balance_sheet,
        'income_statement': income_statement,
        'cashflow': cashflow,
        'market_cap': market_cap,
        'pe_ratio': pe_ratio,
        'eps': eps,
        'industry': industry,
        'description': description,
        'recommendations': recommendations,
        'beta': beta,
        'current_ratio': current_ratio,
        'quick_ratio': quick_ratio,
        'profit_margin': profit_margin,
        'free_cashflow': free_cashflow,
        'earnings_growth': earnings_growth,
        'revenue_growth': revenue_growth,
        'net_profit_margin': net_profit_margin,
        'roe': roe,
        'enterprise_value': enterprise_value,
        'bvps': bvps,
        'major_holders': major_holders,
        'institutional_holders': institutional_holders
    }

def generate_composite_signal(macd, rsi, macd_threshold=0, rsi_threshold=50):
    if macd > macd_threshold and rsi > rsi_threshold:
        return 'Buy'
    elif macd < macd_threshold and rsi < rsi_threshold:
        return 'Sell'
    else:
        return 'Hold'

# 替換行業趨勢的指數代碼
def get_industry_trend(industry_ticker="^TWII"):  # 使用台灣加權指數替代
    industry_data = yf.Ticker(industry_ticker)
    history = industry_data.history(period="6mo")  # 最近六個月的數據

    if len(history) == 0:
        print("數據不足，無法計算行業趨勢。")
        return None

    short_ma = history['Close'].rolling(window=50).mean().iloc[-1]
    long_ma = history['Close'].rolling(window=200).mean().iloc[-1]

    if short_ma > long_ma:
        return 0.8  # 趨勢向上
    else:
        return 0.4  # 趨勢向下


# 簡單基於移動平均線的情緒指標
def get_market_sentiment(index_ticker="^TWII"):
    index_data = yf.Ticker(index_ticker)
    history = index_data.history(period="6mo")  # 最近六個月的數據
    short_ma = history['Close'].rolling(window=20).mean().iloc[-1]
    long_ma = history['Close'].rolling(window=50).mean().iloc[-1]

    if short_ma > long_ma:
        return 0.75  # 市場情緒偏樂觀
    else:
        return 0.25  # 市場情緒偏悲觀


# 計算市值加權平均價格（VWAP）
def calculate_vwap(stock_data):
    cum_price_volume = (stock_data['Adj Close'] * stock_data['Volume']).cumsum()
    cum_volume = stock_data['Volume'].cumsum()
    vwap = cum_price_volume / cum_volume
    return vwap


# 計算累積/分佈線（Accumulation/Distribution Line, AD Line）
def calculate_ad_line(stock_data):
    mfm = ((stock_data['Close'] - stock_data['Low']) - (stock_data['High'] - stock_data['Close'])) / (stock_data['High'] - stock_data['Low'])
    mfv = mfm * stock_data['Volume']
    ad_line = mfv.cumsum()
    return ad_line


# 計算成交量震盪指標（Volume Oscillator）
def calculate_volume_oscillator(stock_data, short_window=14, long_window=28):
    short_vma = stock_data['Volume'].rolling(window=short_window).mean()
    long_vma = stock_data['Volume'].rolling(window=long_window).mean()
    volume_oscillator = ((short_vma - long_vma) / long_vma) * 100
    return volume_oscillator


# 計算 ATR
def calculate_atr(stock_data, window=14):
    high_low = stock_data['High'] - stock_data['Low']
    high_close = (stock_data['High'] - stock_data['Close'].shift()).abs()
    low_close = (stock_data['Low'] - stock_data['Close'].shift()).abs()
    tr = high_low.combine(high_close, max).combine(low_close, max)
    atr = tr.rolling(window=window).mean()
    return atr

# 計算 Parabolic SAR
def calculate_parabolic_sar(stock_data):
    high = stock_data['High']
    low = stock_data['Low']
    # 使用Talib或自定義計算 Parabolic SAR 的邏輯
    # 返回計算結果
    return parabolic_sar

# 計算 Parabolic SAR
def calculate_parabolic_sar(stock_data):
    high = stock_data['High']
    low = stock_data['Low']
    parabolic_sar = ta.SAR(high, low, acceleration=0.02, maximum=0.2)
    return parabolic_sar

# 計算 Stochastic Oscillator
def calculate_stochastic(stock_data, k_window=14, d_window=3):
    low_min = stock_data['Low'].rolling(window=k_window).min()
    high_max = stock_data['High'].rolling(window=k_window).max()
    stoch_k = (stock_data['Close'] - low_min) / (high_max - low_min) * 100
    stoch_d = stoch_k.rolling(window=d_window).mean()
    return stoch_k, stoch_d

# 計算 VWMA
def calculate_vwma(stock_data, window=20):
    volume = stock_data['Volume']
    price = stock_data['Close']
    vwma = (price * volume).rolling(window=window).sum() / volume.rolling(window=window).sum()
    return vwma

# 計算 VROC
def calculate_vroc(stock_data, window=14):
    volume = stock_data['Volume']
    vroc = ((volume - volume.shift(window)) / volume.shift(window)) * 100
    return vroc

# 計算 Momentum
def calculate_momentum(stock_data, window=14):
    momentum = stock_data['Close'].diff(window)
    return momentum

# 計算 Williams %R
def calculate_williams_r(stock_data, period=14):
    high_max = stock_data['High'].rolling(window=period).max()
    low_min = stock_data['Low'].rolling(window=period).min()
    williams_r = (high_max - stock_data['Close']) / (high_max - low_min) * -100
    return williams_r

# 計算 Pivot Points 和 支撐/阻力
def calculate_fibonacci_retracement(stock_data):
    try:
        max_price = stock_data['High'].max()
        min_price = stock_data['Low'].min()

        if pd.isna(max_price) or pd.isna(min_price):
            print("數據缺失，無法計算費波那契回撤位")
            return None

        diff = max_price - min_price
        retracement_levels = {
            '23.6%': max_price - diff * 0.236,
            '38.2%': max_price - diff * 0.382,
            '50%': max_price - diff * 0.5,
            '61.8%': max_price - diff * 0.618,
            '78.6%': max_price - diff * 0.786,
        }
        return retracement_levels
    except Exception as e:
        print(f"計算費波那契回撤位時出錯: {e}")
        return None

# 計算 RVI (Relative Vigor Index)
def calculate_rvi(stock_data, period=10):
    close_open = stock_data['Close'] - stock_data['Open']
    high_low = stock_data['High'] - stock_data['Low']
    rvi = (close_open + 2 * close_open.shift(1) + 2 * close_open.shift(2) + close_open.shift(3)) / (high_low + 2 * high_low.shift(1) + 2 * high_low.shift(2) + high_low.shift(3))
    rvi_signal = rvi.rolling(window=period).mean()
    return rvi_signal

# 計算 MACD
def calculate_macd(close_prices, short_window=12, long_window=26, signal_window=9):
    short_ema = close_prices.ewm(span=short_window, adjust=False).mean()
    long_ema = close_prices.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    macd_signal = macd.ewm(span=signal_window, adjust=False).mean()
    macd_diff = macd - macd_signal
    return macd, macd_signal, macd_diff

# 計算 KD 指標
def calculate_kd(stock_data, period=14, smooth_k=3, smooth_d=3):
    low_min = stock_data['Low'].rolling(window=period).min()
    high_max = stock_data['High'].rolling(window=period).max()
    rsv = (stock_data['Adj Close'] - low_min) / (high_max - low_min) * 100
    k = rsv.ewm(com=smooth_k-1, adjust=False).mean()
    d = k.ewm(com=smooth_d-1, adjust=False).mean()
    return k, d

# 計算布林帶
def calculate_bollinger_bands(close_prices, window=20, num_std=2):
    rolling_mean = close_prices.rolling(window).mean()
    rolling_std = close_prices.rolling(window).std()
    bb_upper = rolling_mean + (rolling_std * num_std)
    bb_lower = rolling_mean - (rolling_std * num_std)
    return bb_upper, bb_lower

# 計算ADX
def calculate_adx(stock_data, window=14):
    plus_dm = stock_data['High'].diff().clip(lower=0)
    minus_dm = stock_data['Low'].diff().clip(upper=0).abs()
    tr = stock_data[['High', 'Low', 'Close']].apply(lambda x: max(x['High'] - x['Low'], abs(x['High'] - x['Close']), abs(x['Low'] - x['Close'])), axis=1)
    atr = tr.rolling(window).mean()
    plus_di = 100 * (plus_dm.rolling(window).mean() / atr)
    minus_di = 100 * (minus_dm.rolling(window).mean() / atr)
    dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
    adx = dx.rolling(window).mean()
    return adx

# 計算Ichimoku指標
def calculate_ichimoku(stock_data):
    high = stock_data['High']
    low = stock_data['Low']
    close = stock_data['Close']

    tenkan_sen = (high.rolling(window=9).max() + low.rolling(window=9).min()) / 2
    kijun_sen = (high.rolling(window=26).max() + low.rolling(window=26).min()) / 2
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)
    senkou_span_b = ((high.rolling(window=52).max() + low.rolling(window=52).min()) / 2).shift(26)
    chikou_span = close.shift(-26)  # 将 Chikou Span 设置为滞后 26 天的收盘价

    return senkou_span_a, senkou_span_b, kijun_sen, tenkan_sen, chikou_span



# 生成報告文本
def format_analysis_text(result):
    fibonacci_retracement = result['fibonacci_retracement']

    fibonacci_text = "無法計算費波那契回撤位" if not isinstance(fibonacci_retracement, dict) else ", ".join([f"{level}: {price:.2f} TWD" for level, price in fibonacci_retracement.items()])

    analysis_text = f"""
    股票分析報告 ({ticker}):

    **股價和技術分析:**
    - 當前價格: {result['current_price']} TWD
    - 購買價格: {result['buy_price']} TWD
    - 最大回撤: {result['max_drawdown']:.2f}%
    - 年化回報率: {result['annual_return']:.2f}%
    - 50日移動均線: {result['50_day_ma']:.2f} TWD
    - 200日移動均線: {result['200_day_ma']:.2f} TWD
    - 相對強弱指數 (RSI): {result['rsi']:.2f}
    - MACD: {result['macd']:.2f}
    - MACD Signal: {result['macd_signal']:.2f}
    - KD 指標 (K值): {result['kd_k']:.2f}
    - KD 指標 (D值): {result['kd_d']:.2f}
    - 布林帶上軌: {result['bb_upper']:.2f} TWD
    - 布林帶下軌: {result['bb_lower']:.2f} TWD
    - ADX: {result['adx']:.2f}
    - Ichimoku 指標 (A): {result['ichimoku_a']:.2f}
    - Ichimoku 指標 (B): {result['ichimoku_b']:.2f}
    - 成交量震盪指標 (Volume Oscillator): {result['volume_oscillator']:.2f}%
    - 累積/分佈線 (AD Line): {result['ad_line']:.2f}
    - 市值加權平均價格 (VWAP): {result['vwap']:.2f} TWD
    - 綜合信號: {result['composite_signal']}
    - 市場情緒: {result['market_sentiment']}
    - 行業趨勢: {result['industry_trend']}

    **公司信息:**
    - 市場資本化: {result['market_cap']:,.0f} TWD
    - 市盈率: {result['pe_ratio']}
    - 每股收益 (EPS): {result['eps']}
    - 行業: {result['industry']}
    - 公司描述: {result['description']}

    **財務比率:**
    - 當前比率: {result['current_ratio']:.2f}
    - 速動比率: {result['quick_ratio']:.2f}
    - 利潤率: {result['profit_margin']:.2f}%

    **自由現金流:**
    - {result['free_cashflow']:,.0f} TWD

    **增長率分析:**
    - 收益增長率: {result['earnings_growth']:.2f}%
    - 收入增長率: {result['revenue_growth']:.2f}%

    **淨利潤率:**
    - {result['net_profit_margin']}

    **股東權益回報率 (ROE):**
    - {result['roe']}

    **企業價值 (EV):**
    - {result['enterprise_value']:,.0f} TWD

    **每股淨資產 (BVPS):**
    - {result['bvps']}

    **財務報表摘要:**
    - 資產負債表: {result['balance_sheet'].head()}
    - 損益表: {result['income_statement'].head()}
    - 現金流量表: {result['cashflow'].head()}

    **股息和拆股信息:**
    - 股息: {result['dividends'].tail()}
    - 拆股: {result['splits']}

    **分析師推薦:**
    - {result['recommendations'].tail()}

    **股東信息:**
    - 主要持股人: {result['major_holders']}
    - 機構持股人: {result['institutional_holders']}

    **貝塔係數 (Beta):**
    - {result['beta']}

    **費波那契回撤位:**
    - {fibonacci_text}

    **技術指標:**
    - Parabolic SAR: {result['parabolic_sar']:.2f} TWD
    - 平均真實波幅 (ATR): {result['atr']:.2f}
    - 隨機震盪指標 (Stochastic Oscillator) K值: {result['stochastic_k']:.2f}
    - 隨機震盪指標 (Stochastic Oscillator) D值: {result['stochastic_d']:.2f}
    - 成交量加權移動平均線 (VWMA): {result['vwma']:.2f} TWD
    - 成交量變動率 (VROC): {result['vroc']:.2f}%
    - 動量指標 (Momentum): {result['momentum']:.2f}
    - 威廉指標 (Williams %R): {result['williams_r']:.2f}
    - Pivot Point: {result['pivot_point']:.2f} TWD
    - 支撐位 1 (S1): {result['support_1']:.2f} TWD
    - 支撐位 2 (S2): {result['support_2']:.2f} TWD
    - 阻力位 1 (R1): {result['resistance_1']:.2f} TWD
    - 阻力位 2 (R2): {result['resistance_2']:.2f} TWD
    - 相對活力指標 (RVI): {result['rvi']:.2f}
    """
    return textwrap.dedent(analysis_text).strip()



# 使用 Gemini API 生成報告
def generate_stock_analysis_with_gemini(api_key, analysis_text):
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-flash')
        full_prompt = f"你是一個專業股票分析師，參照這些資料給投資者目前最適合的建議策略，注意使用者購入價格(提醒使用者止損)，目前價格是賣或是進買，使用繁體中文回答：\n\n{analysis_text}"
        response = model.generate_content(contents=[full_prompt])
        return response.candidates[0].content.parts[0].text
    except Exception as e:
        return f"無法生成報告: {e}"

# 主程序
result = analyze_stock(ticker, buy_price)
if result:
    analysis_text = format_analysis_text(result)
    gemini_analysis = generate_stock_analysis_with_gemini(api_key, analysis_text)
    full_report = f"{analysis_text}\n\nGemini API 生成的分析報告:\n{gemini_analysis}" if gemini_analysis else "無法生成Gemini分析報告。"
    print(full_report)
